# RAG Based on HyDE
- Hybrid Search
- ReRanker
- HyDE (Hypothetical Document Embeddings)

## Setting
 - Auto Reload
 - path for utils

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "../../.."
sys.path.append(os.path.abspath(module_path))

## 1. Bedrock Client 생성

In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [4]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

### LLM 로딩 (Claude-v2)

In [5]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [6]:
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 512
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f0d6030c790>, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 512}, streaming=True, callbacks=[<langchain.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f0d5fc8b520>])

### Embedding 모델 선택

In [7]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

In [8]:
def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    
    if is_bedrock_embeddings:
        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings
        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )        
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")
    
    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [9]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)   

Bedrock Embeddings Model Loaded


## 3. Depoly ReRanker model (if needed)

In [10]:
import json
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [11]:
depoly = False

In [12]:
if depoly:

    try:
        role = sagemaker.get_execution_role()
    except ValueError:
        iam = boto3.client('iam')
        role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

    # Hub Model configuration. https://huggingface.co/models
    hub = {
        'HF_MODEL_ID':'BAAI/bge-reranker-large',
        'HF_TASK':'text-classification'
    }

    # create Hugging Face Model Class
    huggingface_model = HuggingFaceModel(
        transformers_version='4.26.0',
        pytorch_version='1.13.1',
        py_version='py39',
        env=hub,
        role=role, 
    )

    # deploy model to SageMaker Inference
    predictor = huggingface_model.deploy(
        initial_instance_count=1, # number of instances
        instance_type='ml.g5.xlarge' # instance type
    )

    print(f'Accept: {predictor.accept}')
    print(f'ContentType: {predictor.content_type}')
    print(f'Endpoint: {predictor.endpoint}')

## 4. Invocation (prediction)

In [13]:
runtime_client = boto3.Session().client('sagemaker-runtime')
print (f'runtime_client: {runtime_client}')

runtime_client: <botocore.client.SageMakerRuntime object at 0x7f0d4c6069e0>


In [14]:
#endpoint_name = "huggingface-pytorch-inference-2023-11-15-04-37-45-120" # ml.m5.2xlarge
endpoint_name = "huggingface-pytorch-inference-2023-11-15-07-53-21-605" # ml.g5.xlarge
deserializer = "application/json"

In [15]:
payload = json.dumps(
    {
        "inputs": [
            {"text": "I hate you", "text_pair": "I don't like you"},
            {"text": "He hates you", "text_pair": "He like you"}
        ]
    }
)

In [16]:
%%time
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Accept=deserializer,
    Body=payload
)
## deserialization
out = json.loads(response['Body'].read().decode()) ## for json
print (f'Response: {out}')

Response: [{'label': 'LABEL_0', 'score': 0.999653697013855}, {'label': 'LABEL_0', 'score': 0.025255175307393074}]
CPU times: user 20.9 ms, sys: 303 µs, total: 21.2 ms
Wall time: 92.4 ms


## 5. LangChainmOpenSearch VectorStore 정의
### 선수 조건
- 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.
#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [17]:
from utils.proc_docs import get_parameter

In [18]:
ssm = boto3.client("ssm", "us-east-1")

opensearch_domain_endpoint = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_domain_endpoint',
)

opensearch_user_id = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_userid',
)

opensearch_user_password = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_password',
)

In [19]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

### Index 이름 셋팅
- 이전 노트북 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 생성된 OpenSearch Index name 입력

In [20]:
index_name = "genai-demo-knox-v1"

### OpenSearch Client 생성

In [21]:
from utils.opensearch import opensearch_utils

In [22]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

### LangChain OpenSearch VectorStore 생성

In [23]:
from langchain.vectorstores import OpenSearchVectorSearch

In [24]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)
vector_db

# 아래수정

## 6. Retriever based on Hybrid Search + HyDE + ReRanker
- LangChain에서 제공하는 **BaseRetriever** 클래스를 상속받아 **Custom Retriever**를 정의 할 수 있습니다.
- Hybrid-Search에 대한 자세한 내용는 **"01_rag_hybrid_search.ipyno"** 에서 확인 가능합니다.
- [RAG-Fusion](https://towardsdatascience.com/forget-rag-the-future-is-rag-fusion-1147298d8ad1)
![rag-fusion.png](../../../imgs/rag-fusion.png)

In [25]:
from utils.rag import OpenSearchHybridSearchRetriever

- 필터 설정 예시
- filter=[ <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your first keyword**"}}, <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your second keyword**"}},<BR>
]

In [26]:
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    os_client=os_client,
    vector_db=vector_db,
    index_name=index_name,

    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for search
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[.5, .5], # [for lexical, for semantic], Lexical, Semantic search 결과에 대한 최종 반영 비율 정의
    reranker=True, # enable reranker with reranker model
    reranker_endpoint_name=endpoint_name, # endpoint name for reranking model
    hyde=True, # enable hyde
    hyde_query=["web_search"], # query type in hyde 
    llm_text=llm_text, # llm for query augmentation in both rag_fusion and HyDE

    # option for async search
    async_mode=True,

    # option for output
    k=5, # 최종 Document 수 정의
    verbose=True,
)

### Retrieval example
- default search

In [27]:
from utils.rag import show_context_used

In [28]:
query = "what is verify DM"

In [29]:
%%time
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print (f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)

 Here is a concise passage answering the question "what is verify DM":

Verify DM is a process used on social media platforms like Twitter to verify the authenticity of accounts belonging to public figures, celebrities, businesses, and organizations. When an account gets verified, it receives a blue checkmark icon next to the account name, indicating that the social media platform has confirmed the identity of the account holder. The verification process typically requires the account owner to provide identifying documents and information to prove they are who they claim to be. Verification helps distinguish official accounts from impersonators or fake accounts, allowing users to be confident that public figures and organizations they interact with online are legitimate. Overall, account verification aims to limit misinformation by authenticating influential users on social media.===== HyDE Answers =====
['what is verify DM', ' Here is a concise passage answering the question "what is 

- update parameters

In [30]:
opensearch_hybrid_retriever.update_search_params(
    k=10,
    minimum_should_match=30,
    filter=[],
    reranker=True,
    reranker_endpoint_name=endpoint_name,
    hyde=True, # enable hyde
    hyde_query=["web_search"], # query type in hyde 
    llm_text=llm_text, # llm for query augmentation in rag_fusion
    verbose=False
)

In [31]:
query = "how to vefify DM"
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print(f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)

 Here is a concise passage answering how to verify a direct message (DM):

To verify a direct message (DM) on social media or messaging platforms, there are a few steps you can take:

- Check the sender's profile and handle to confirm it matches the person or account you think sent the DM. Verify it's not an impersonator account.

- Look for context clues in the message content that reflect your previous conversations or relationship with the sender. This helps confirm the message is from someone you actually know.

- If you're still uncertain, reply to the message asking the sender to confirm their identity. Ask a question only the real sender would know. 

- You can also try contacting the sender through a different platform to verify they sent the initial DM. 

- For high-risk conversations, consider voice or video calling the sender to fully authenticate their identity. 

- Enable DM privacy settings and be wary of opening DMs from accounts you can't verify are legitimate. Ignore o

## 5. RAG using RetrievalQA powered by LangChain

In [32]:
from utils.rag import run_RetrievalQA
from langchain.prompts import PromptTemplate

### Prompting
- [TIP] Prompt의 instruction의 경우 한글보다 영어로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [33]:
prompt_template = """
\n\nHuman: Here is the context, inside <context></context> XML tags.

<context>
{context}
</context>

Only using the context as above, answer the following question with the rules as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.

Question:
{question}

If the answer is not in the context, just say "Could not find answer in given contexts."

\n\nAssistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

### Update Search Params (Optional)

In [34]:
from langchain.chains import RetrievalQA

In [35]:
opensearch_hybrid_retriever.update_search_params(
    k=3,
    minimum_should_match=3,
    filter=[],
    reranker=True,
    reranker_endpoint_name=endpoint_name,
    hyde=True, # enable hyde
    hyde_query=["web_search"], # query type in hyde 
    llm_text=llm_text, # llm for query augmentation in rag_fusion
    verbose=False
)

### Request

In [36]:
qa = RetrievalQA.from_chain_type(
    llm=llm_text,
    chain_type="stuff",
    retriever=opensearch_hybrid_retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": PROMPT,
        "verbose": False,
    },
    verbose=False
)

In [37]:
query = "How does RKP protect critical kernel data structures from modification?"
response = qa(query)

 Here is a concise passage answering how RKP protects critical kernel data structures from modification:

RKP (Real-time Kernel Protection) is a security module in Android that aims to protect the kernel by preventing unauthorized modifications to critical data structures. It does this by making certain kernel data structures read-only, restricting access to sensitive kernel code, and adding additional checks around structures that can't be made read-only. For example, RKP marks structures like the syscall table and credentials as read-only after initialization. It also hooks certain kernel functions like copy_to_user() to prevent overwrite attacks on key data. By proactively hardening the kernel in this manner, RKP reduces the kernel's attack surface and makes it more difficult for malware to tamper with critical kernel data structures. RKP protects critical kernel data structures from modification by monitoring some critical kernel data structures to verify that they are not exploite

In [38]:
print("##################################")
print("query: ", query)
print("##################################")

print (colored("\n\n### Answer ###", "blue"))
print_ww(response['result'])

print (colored("\n\n### Contexts ###", "green"))
show_context_used(response['source_documents'])

##################################
query:  How does RKP protect critical kernel data structures from modification?
##################################


### Answer ###
 RKP protects critical kernel data structures from modification by monitoring some critical kernel
data structures to verify that they are not exploited. In particular, RKP protects the data that
defines the credentials assigned to running user processes to prevent attackers from modifying this
data.


### Contexts ###
-----------------------------------------------
1. Chunk: 244 Characters
-----------------------------------------------
 Third, RKP monitors some critical kernel data structures to verify that they are not exploited.
In particular, RKP protects the data that de nes the credentials assigned to running user processes
to prevent attackers from modifying this data.
metadata:
 {'tag': 'para', 'row': 113, 'id': '1922c223-7d95-453a-9c56-043b19d98be0'}
-----------------------------------------------
2. Chunk: 385